In [ ]:
# This notebook illustrates palindrome case 
# unpack prepared combinatorics dataset from palindromes.zip in the directory containing this
# notebook

import os, shutil
import numpy as np
import keras
from keras.models import Sequential

partition = {}
labels = {}

base_dir = './palindromes'
try:
    os.mkdir(base_dir)
except:
    pass

train_dir = os.path.join(base_dir,'train')

try:
    os.mkdir(train_dir)
except:
    pass

train_palindromes_dir = os.path.join(train_dir, 'palindromes')
try:
    os.mkdir(train_palindromes_dir)
except:
    pass

fnames = ['pdata{}.npz'.format(i) for i in range(1000)]
partition['train'] = fnames


for fname in fnames:
    src = os.path.join('./', fname)
    dst = os.path.join(train_palindromes_dir, fname)
    shutil.copyfile(src, dst) 
    
train_nonpalindromes_dir = os.path.join(train_dir, 'nonpalindromes')
try:
    os.mkdir(train_nonpalindromes_dir)
except:
    pass


fnames = ['data{}.npz'.format(i) for i in range(1000)]

partition['train'] += fnames


for fname in fnames:
    src = os.path.join('./', fname)
    dst = os.path.join(train_nonpalindromes_dir, fname)
    shutil.copyfile(src, dst) 
    
test_dir = os.path.join(base_dir,'test')

try:
    os.mkdir(test_dir)
except:
    pass

test_palindromes_dir = os.path.join(test_dir, 'palindromes')
try:
    os.mkdir(test_palindromes_dir)
except:
    pass

fnames = ['pdata{}.npz'.format(i) for i in range(1000,1500)]
partition['test'] = fnames




for fname in fnames:
    src = os.path.join('./', fname)
    dst = os.path.join(test_palindromes_dir, fname)
    shutil.copyfile(src, dst) 
    
test_nonpalindromes_dir = os.path.join(test_dir, 'nonpalindromes')
try:
    os.mkdir(test_nonpalindromes_dir)
except:
    pass

fnames = ['data{}.npz'.format(i) for i in range(1000,1500)]
partition['test'] += fnames


for fname in fnames:
    src = os.path.join('./', fname)
    dst = os.path.join(test_nonpalindromes_dir, fname)
    shutil.copyfile(src, dst) 
    
valid_dir = os.path.join(base_dir,'validate')

try:
    os.mkdir(valid_dir)
except:
    pass

valid_palindromes_dir = os.path.join(valid_dir, 'palindromes')
try:
    os.mkdir(valid_palindromes_dir)
except:
    pass

fnames = ['pdata{}.npz'.format(i) for i in range(1500,2000)]
partition['validate'] = fnames


for fname in fnames:
    src = os.path.join('./', fname)
    dst = os.path.join(valid_palindromes_dir, fname)
    shutil.copyfile(src, dst) 
    
valid_nonpalindromes_dir = os.path.join(valid_dir, 'nonpalindromes')
try:
    os.mkdir(valid_nonpalindromes_dir)
except:
    pass

fnames = ['data{}.npz'.format(i) for i in range(1500,2000)]
partition['validate'] += fnames


for fname in fnames:
    src = os.path.join('./', fname)
    dst = os.path.join(valid_nonpalindromes_dir, fname)
    shutil.copyfile(src, dst) 
    

labels = {}
for part in partition:
    for el in partition[part]:
        if 'pdata' in el:
            labels[el] = 1
        else:
            labels[el] = 0

In [ ]:
from keras.utils import np_utils

class DataGenerator(keras.utils.all_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(210,210), n_channels=210,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            if 'p' == str(ID)[0]:
                src = os.path.join(train_palindromes_dir, str(ID))
            else:
                src = os.path.join(train_nonpalindromes_dir, str(ID))
            X[i,] = np.load(src)['arr_0']

            # Store class
            y[i] = self.labels[ID]

        return X, y

class ValidateGenerator(keras.utils.all_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(210,210), n_channels=210,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            if 'p' == str(ID)[0]:
                src = os.path.join(valid_palindromes_dir, str(ID))
            else:
                src = os.path.join(valid_nonpalindromes_dir, str(ID))
            X[i,] = np.load(src)['arr_0']

            # Store class
            y[i] = self.labels[ID]

        return X, y

In [ ]:
training_generator = DataGenerator(partition['train'], labels)
validation_generator = ValidateGenerator(partition['validate'], labels)

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape = (210,210,210)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(8, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=1e-4),
             metrics=['acc'])

In [ ]:
history = model.fit(
    training_generator,
    steps_per_epoch=30,
    epochs=50,
    validation_data=validation_generator,
validation_steps=30)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.figure()
plt.show()